In [1]:
import os
import getpass
from langchain import hub
from langchain_chroma import Chroma
from langchain_cohere import ChatCohere, CohereEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [2]:
os.environ['COHERE_API_KEY'] = getpass.getpass()

 ········


In [3]:
cohere_api_key = os.environ['COHERE_API_KEY']

In [4]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = getpass.getpass()

 ········


In [5]:
loader = PDFPlumberLoader('https://arxiv.org/pdf/2405.10725')
docs = loader.load()

In [6]:
# total 12 documents
print(docs)

[Document(page_content='INDUS: Effective and Efficient Language Models for Scientific Applications\nBishwaranjanBhattacharjee1,AashkaTrivedi1,MasayasuMuraoka1,\nMuthukumaranRamasubramanian3,TakumaUdagawa1,IkshaGurung3,RongZhang1,\nBharathDandala1,RahulRamachandran2,ManilMaskey2,KaylinBugbee2,MikeLittle4,\nElizabethFancher2,LaurenSanders5,SylvainCostes5,SergiBlanco-Cuaresma6,KellyLockhart6,\nThomasAllen6,FelixGrezes6,MeganAnsdell7,AlbertoAccomazzi6,YousefEl-Kurdi1,\nDavisWertheimer1,BirgitPfitzmann1,CesarBerrospiRamis1,MicheleDolfi1,RafaelTeixeiradeLima1,\nPanagiotisVagenas1,S.KarthikMukkavilli1,PeterStaar1,SanazVahidinia7,RyanMcGranaghan8,\nArminMehrabian9,TsendgarLee7\n1IBMResearchAI,2 NASAMFSC,3 UAH,4 Navteca,5 NASAAmes,6 Harvard-SmithsonianCfA,\n7 NASAHQ,8 JPL,9 NASAGSFC\nAbstract generation tasks. Most popular LLMs rely on the\ntransformer architecture (Vaswani et al., 2017)\nLargelanguagemodels(LLMs)trainedongen- and are trained using general-purpose corpora\neral domain corpora s

In [7]:
print(docs[0].page_content)

INDUS: Effective and Efficient Language Models for Scientific Applications
BishwaranjanBhattacharjee1,AashkaTrivedi1,MasayasuMuraoka1,
MuthukumaranRamasubramanian3,TakumaUdagawa1,IkshaGurung3,RongZhang1,
BharathDandala1,RahulRamachandran2,ManilMaskey2,KaylinBugbee2,MikeLittle4,
ElizabethFancher2,LaurenSanders5,SylvainCostes5,SergiBlanco-Cuaresma6,KellyLockhart6,
ThomasAllen6,FelixGrezes6,MeganAnsdell7,AlbertoAccomazzi6,YousefEl-Kurdi1,
DavisWertheimer1,BirgitPfitzmann1,CesarBerrospiRamis1,MicheleDolfi1,RafaelTeixeiradeLima1,
PanagiotisVagenas1,S.KarthikMukkavilli1,PeterStaar1,SanazVahidinia7,RyanMcGranaghan8,
ArminMehrabian9,TsendgarLee7
1IBMResearchAI,2 NASAMFSC,3 UAH,4 Navteca,5 NASAAmes,6 Harvard-SmithsonianCfA,
7 NASAHQ,8 JPL,9 NASAGSFC
Abstract generation tasks. Most popular LLMs rely on the
transformer architecture (Vaswani et al., 2017)
Largelanguagemodels(LLMs)trainedongen- and are trained using general-purpose corpora
eral domain corpora showed remarkable re-
like Wikipedia or

In [8]:
print(docs[0].metadata)

{'source': 'https://arxiv.org/pdf/2405.10725', 'file_path': 'https://arxiv.org/pdf/2405.10725', 'page': 0, 'total_pages': 12, 'Author': '', 'CreationDate': 'D:20240522001021Z', 'Creator': 'LaTeX with hyperref', 'Keywords': '', 'ModDate': 'D:20240522001021Z', 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'Producer': 'pdfTeX-1.40.25', 'Subject': '', 'Title': '', 'Trapped': 'False'}


In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, add_start_index = True)
doc_splits = text_splitter.split_documents(documents=docs)

In [10]:
# total 57 chunks
print(doc_splits)

[Document(page_content='INDUS: Effective and Efficient Language Models for Scientific Applications\nBishwaranjanBhattacharjee1,AashkaTrivedi1,MasayasuMuraoka1,\nMuthukumaranRamasubramanian3,TakumaUdagawa1,IkshaGurung3,RongZhang1,\nBharathDandala1,RahulRamachandran2,ManilMaskey2,KaylinBugbee2,MikeLittle4,\nElizabethFancher2,LaurenSanders5,SylvainCostes5,SergiBlanco-Cuaresma6,KellyLockhart6,\nThomasAllen6,FelixGrezes6,MeganAnsdell7,AlbertoAccomazzi6,YousefEl-Kurdi1,\nDavisWertheimer1,BirgitPfitzmann1,CesarBerrospiRamis1,MicheleDolfi1,RafaelTeixeiradeLima1,\nPanagiotisVagenas1,S.KarthikMukkavilli1,PeterStaar1,SanazVahidinia7,RyanMcGranaghan8,\nArminMehrabian9,TsendgarLee7\n1IBMResearchAI,2 NASAMFSC,3 UAH,4 Navteca,5 NASAAmes,6 Harvard-SmithsonianCfA,\n7 NASAHQ,8 JPL,9 NASAGSFC\nAbstract generation tasks. Most popular LLMs rely on the\ntransformer architecture (Vaswani et al., 2017)\nLargelanguagemodels(LLMs)trainedongen- and are trained using general-purpose corpora\neral domain corpora s

In [11]:
vectorstore = Chroma.from_documents(documents=doc_splits, embedding=CohereEmbeddings())
retriever = vectorstore.as_retriever()

In [12]:
model = ChatCohere(model='command-r')

In [13]:
prompt = hub.pull('rlm/rag-prompt')

In [14]:
output_parser = StrOutputParser()

In [15]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

format_docs(docs)

'INDUS: Effective and Efficient Language Models for Scientific Applications\nBishwaranjanBhattacharjee1,AashkaTrivedi1,MasayasuMuraoka1,\nMuthukumaranRamasubramanian3,TakumaUdagawa1,IkshaGurung3,RongZhang1,\nBharathDandala1,RahulRamachandran2,ManilMaskey2,KaylinBugbee2,MikeLittle4,\nElizabethFancher2,LaurenSanders5,SylvainCostes5,SergiBlanco-Cuaresma6,KellyLockhart6,\nThomasAllen6,FelixGrezes6,MeganAnsdell7,AlbertoAccomazzi6,YousefEl-Kurdi1,\nDavisWertheimer1,BirgitPfitzmann1,CesarBerrospiRamis1,MicheleDolfi1,RafaelTeixeiradeLima1,\nPanagiotisVagenas1,S.KarthikMukkavilli1,PeterStaar1,SanazVahidinia7,RyanMcGranaghan8,\nArminMehrabian9,TsendgarLee7\n1IBMResearchAI,2 NASAMFSC,3 UAH,4 Navteca,5 NASAAmes,6 Harvard-SmithsonianCfA,\n7 NASAHQ,8 JPL,9 NASAGSFC\nAbstract generation tasks. Most popular LLMs rely on the\ntransformer architecture (Vaswani et al., 2017)\nLargelanguagemodels(LLMs)trainedongen- and are trained using general-purpose corpora\neral domain corpora showed remarkable re-\nl

In [16]:
chain = (
    {'context' : retriever | format_docs, 'question' : RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [17]:
chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'CohereEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x140161210>)
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])
| ChatCohere(client=<cohere.client.Client object at 0x131129450>, async_client=<cohere.client.AsyncClient object at 0x140610350>, model='command-r', cohere_api_key=SecretStr('**********'))
| StrOutputParser()

In [18]:
chain.invoke('Can you brief me the summary of this research paper in a very plain english which is easy to understand ?')

'The research focuses on developing INDUS, a series of encoder-based models designed to aid understanding in specific domains, namely Earth science. These models are trained using specialized corpora. According to the research, INDUS outperforms comparable models, and its smaller version, INDUS SMALL, also performs relatively well.'

Now, I have a follow-up question to understand about encoder models, LLM, everything about INDUS. How did it prove to be better than the other models?

But for this, it would be ideal if the model knows about the conversation history so it can boil down the exact question very precisely and become time efficient.

Thus let's add a chat history to this RAG Chain.

##### 2 things : 
##### a) contextualize input question based on chat history
##### b) add chat history to prompt

In [19]:
contextualize_q_system_prompt = "Based on the given chat history and the most recent user question which\
could reference context in the chat history, create a standalone question which could be understood without\
the chat history. Do not answer the question. Just reformulate it if needed otherwise return it as it is."

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ('system', contextualize_q_system_prompt),
    MessagesPlaceholder('chat_history'),
    ('human', '{input}')
])

history_aware_retriever = create_history_aware_retriever(model, retriever, contextualize_q_prompt)

In [20]:
qa_system_prompt = "You are an assistant for question answering tasks. Use the retrieved context to answer\
the user input. If you don't know the answer, just say you do not know.\
{context}"

qa_prompt = ChatPromptTemplate.from_messages([
    ('system', qa_system_prompt),
    MessagesPlaceholder('chat_history'),
    ('human', '{input}')
])

question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

In [21]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [22]:
chat_history = []
chat_history.extend([
    HumanMessage(content='Can you brief me the summary of this research paper in a very plain english which is easy to understand ?'),
    AIMessage(content='This paper introduces INDUS, a family of encoder-based LLMs designed for use in the science domain. It compares their performance with other models and finds that INDUS outperformed them all. The researchers also created smaller versions of the models for resource-constrained applications.')
])

chat_history

[HumanMessage(content='Can you brief me the summary of this research paper in a very plain english which is easy to understand ?'),
 AIMessage(content='This paper introduces INDUS, a family of encoder-based LLMs designed for use in the science domain. It compares their performance with other models and finds that INDUS outperformed them all. The researchers also created smaller versions of the models for resource-constrained applications.')]

In [23]:
question = "what are encoder models?"
response = rag_chain.invoke({'input' : question, 'chat_history' : chat_history})
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response['answer'])
])
chat_history

[HumanMessage(content='Can you brief me the summary of this research paper in a very plain english which is easy to understand ?'),
 AIMessage(content='This paper introduces INDUS, a family of encoder-based LLMs designed for use in the science domain. It compares their performance with other models and finds that INDUS outperformed them all. The researchers also created smaller versions of the models for resource-constrained applications.'),
 HumanMessage(content='what are encoder models?'),
 AIMessage(content='Encoder models are a type of neural network architecture commonly used in natural language processing (NLP) and speech processing tasks. They are called "encoders" because they transform input data, such as a sentence or a sound clip, into a smaller set of values, known as an encoding. This encoding captures the most important aspects of the original data.\n\nIn the context of NLP, encoders read and comprehend text, then summarize or encode the content into fixed-length vectors,

In [24]:
question = "what is an encoder based LLM? Tell me about its architecture."
response = rag_chain.invoke({'input' : question, 'chat_history' : chat_history})
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response['answer'])
])
chat_history

[HumanMessage(content='Can you brief me the summary of this research paper in a very plain english which is easy to understand ?'),
 AIMessage(content='This paper introduces INDUS, a family of encoder-based LLMs designed for use in the science domain. It compares their performance with other models and finds that INDUS outperformed them all. The researchers also created smaller versions of the models for resource-constrained applications.'),
 HumanMessage(content='what are encoder models?'),
 AIMessage(content='Encoder models are a type of neural network architecture commonly used in natural language processing (NLP) and speech processing tasks. They are called "encoders" because they transform input data, such as a sentence or a sound clip, into a smaller set of values, known as an encoding. This encoding captures the most important aspects of the original data.\n\nIn the context of NLP, encoders read and comprehend text, then summarize or encode the content into fixed-length vectors,

In [25]:
response['context']

[Document(page_content='Model Specifications We created our sentence approximately56Msentences. Thismaskedauto-\nembeddingmodelbyfine-tuningINDUS . Here- encoder model consisted of a full encoder along\nBASE\nafter, we refer to the resulting retriever model as with a shallow decoder. The model uses masked\nINDUS-RETRIEVER . Wefollowedabi-encoder languagemodelingwithatrainingobjectivetore-\nBASE\nframework(ReimersandGurevych,2019),andex- covertheoriginalsentencebasedonthedecoder’s\nperimented with multiple pooling strategies and maskedinputandthesentenceembeddinggener-\nfoundthatthemeanpoolingofthecontextualized atedfromtheencoder’smaskedinput,viamasked\ntransformerrepresentationsperformedthebest. language modelling. There is no distillation loss\ncontributingtothisstep,whichcanbeviewedasan\nTraining Details We trained each stage on 2\nextendedpretrainingmechanism. Wefindthatthe\nA100GPUswithaneffectivebatchsizeof1,024.\nRetroMAEpretrainingdoesnotgiveusgoodgains\nWe first trained with u

Now we can see that it's easy to ask follow-up questions to our assistant by attaching a conversational chat history for it to refer before giving an answer

But one problem with this system is the effort needed to attach each question-answer response to this chat history after each invocation call. 

So to counter this, we need to modify it in a way that chat history gets updated after every invocation automatically.

##### Adding message history using RunnableWithMessageHistory to store messages based on session
##### For now, storing it in-memory using a dictionary with session id and an instance of BaseChatMessageHistory

In [26]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history',
    output_messages_key='answer'
)

In [27]:
conversational_rag_chain.input_schema.schema()

{'title': 'RunnableWithChatHistoryInput',
 'type': 'object',
 'properties': {'input': {'title': 'Input',
   'anyOf': [{'type': 'string'},
    {'$ref': '#/definitions/BaseMessage'},
    {'type': 'array', 'items': {'$ref': '#/definitions/BaseMessage'}}]}},
 'required': ['input'],
 'definitions': {'BaseMessage': {'title': 'BaseMessage',
   'description': 'Base abstract Message class.\n\nMessages are the inputs and outputs of ChatModels.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'title': 'Type', 'type': 'string'},
    'name': {'title': 'Name', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['content', 'type']}}}

In [28]:
conversational_rag_chain.output_schema.schema()

{'title': 'RunnableSequenceOutput',
 'type': 'object',
 'properties': {'chat_history': {'title': 'Chat History',
   'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}},
  'input': {'title': 'Input', 'type': 'string'},
  'answer': {'title': 'Answer', 'type': 'string'}},
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id

In [29]:
response = conversational_rag_chain.invoke(
    {'input': 'Summarize the research paper for me.'},
    config={
        'configurable': {
            'session_id': 'chat1'
        }
    },
)

In [30]:
response

{'input': 'Summarize the research paper for me.',
 'chat_history': [],
 'context': [Document(page_content='domains. Specifically, we created a new manu-\ninformation retrieval systems and mimic the real\nallyannotateddatasetCLIMATE-CHANGE NER8,in\nworlddata,Wecombined26,839randomADSab-\nwhichthenamedentitiesofinterestoriginatefrom\nstracts with these annotated paragraphs. On an\ncomplextaxonomiesusedinclimate-relatedlitera-\naverage, each query is 12 words long, and each\nture. Thisdatasetcomprises534abstractssourced\nparagraphis120wordslong. WeusedRecall@10\nfromSemanticScholarAcademicGraph(Kinney\nasevaluationmetricsinceeachquestionhasonly\netal.,2023),collectedusingaseedsetofclimate-\nonerelevantdocument.\nrelated keywords such as wildfire or floods. The\nabstractswereannotatedusingtheIOB(inside,out-\n6 ExperimentalResults\nside,beginning)taggingschemeandencompasses\nadiversearrayofentitytypes,asshowninTable4. Baselines Wecompared INDUS modelsagainst\nopensourcemodelsofsimilarsizes:

In [31]:
print(response['answer'])

The research paper introduces NASA-QA9, a new dataset for developing and evaluating NLP models tailored to understanding climate-related topics. This dataset is manually annotated with questions that can be answered from each paragraph, making it useful for information retrieval and question-answering tasks. The paper also introduces a comprehensive climate change dataset with complex taxonomies, which is useful for improving climate literature analysis. 

To evaluate the datasets, the researchers compared the performance of various open-source models, including RoBERTa, SCIBERT, and MINILM, using metrics like Recall@10. The results show that a domain-specific teacher model, INDUS, outperforms the baseline models in most cases, suggesting the effectiveness of knowledge distillation in this domain. 

Overall, this work aims to bridge the gap in climate-related NLP research by providing new datasets and evaluation methods for developing better models to address climate change-related que

In [32]:
response = conversational_rag_chain.invoke(
    {'input': 'What is NASA-QA9? Give me all the details about this and how it is being used in the research paper?'},
    config={
        'configurable': {
            'session_id': 'chat1'
        }
    },
)

In [33]:
(response['input'], response['answer'])

('What is NASA-QA9? Give me all the details about this and how it is being used in the research paper?',
 "NASA-QA9 is a large-scale dataset introduced in the research paper as a benchmark for climate-related question answering. It is designed to develop and evaluate extractive question-answering models that can comprehend and provide answers from climate change-related paragraphs. This dataset is a modification and enhancement of the previous NASA-QA dataset. \n\nHere are the key details about NASA-QA9 and its usage in the research:\n1. Dataset Creation: NASA-QA9 is created by sampling paragraphs from various sources, including AGU, AMS, ADS, PMC, and PubMed. These paragraphs are manually annotated with questions that can be answered based on the information within each paragraph. \n2. Sample Size: The dataset contains a total of 686 paragraphs, which are divided into a training set (398 paragraphs) and a validation set (288 paragraphs). Each paragraph is, on average, 120 words long. 

In [34]:
response = conversational_rag_chain.invoke(
    {'input': 'What is the INDUS model? Cover all the details about it from its initiation phase, to architecture and all its specifications.'},
    config={
        'configurable': {
            'session_id': 'chat1'
        }
    },
)

In [35]:
(response['input'], response['answer'])

('What is the INDUS model? Cover all the details about it from its initiation phase, to architecture and all its specifications.',
 'The INDUS model is a family of transformer-based language models developed specifically for scientific applications. The name "INDUS" stands for "Effective and Efficient Language models for Scientific Applications." These models are designed to address the challenges of understanding and extracting information from scientific texts. \n\nHere\'s a comprehensive overview of the INDUS model:\n1. Initiation: The researchers started by curating a large corpus of scientific text, which included papers from various domains like computer science, biology, medicine, and climate change. This corpus was used to train the INDUS models, focusing on creating domain-specific representations. \n2. Data Processing: The scientific corpus underwent pre-processing using a customized byte-pair encoding (BPE) tokenizer called INDUSBPE. This tokenizer is trained on the scientif

In [44]:
print(response['answer'])

The INDUS model is a pivotal component of the research, designed specifically for scientific domain tasks. Let's delve into the details of its development and architecture.

Initiation Phase:
INDUS was developed with a focus on creating a domain-specific language model tailored to scientific literature. The model's training leveraged a large corpus of scientific papers and articles, ensuring its exposure to the specific vocabulary, terminology, and linguistic nuances prevalent in this domain. 

This curated scientific corpus, used for training INDUS, is compiled from multiple sources, including arXiv, a repository of electronic preprints, and domain-specific datasets like PubMed and PubMed Central, which contain a vast collection of biomedical research articles.

Architecture and Specifications:
INDUS is built upon the foundation of the transformer-based architecture, sharing similarities with other state-of-the-art models like RoBERTa and BERT. It employs the encoder component of the 